- 깊은 신경망을 구성하는 것은 매우 어려운 일 ㅠㅠ
    - 까다로운 **그래디언트 소실**혹은 **그래디언트 폭주** 문제에 젝면할 수 있음
    - 데이터 자체가 부족하지 않거나, 데이터를 레이블링하는 것 자체가 큰 코스트일 수 있음
    - 훈련 속도가 극단적으로 느려질 수 있음
    - 오버피팅의 가능성이 큼

# 11.1. 그래디언트 소실과 폭주 문제
- 역전파 알고리즘은 출력층에서 입력층으로 오차 그래디언트를 전파하고, 알고리즘이 오차함수의 그래디언트를 계산하면, 경사하강 단계에서 이 그래디언트를 이용하여 각 파라미터를 수정하는 원리
    - 그런데 이 떄 하위층으로 알고리즘이 진행될 수록 그래디언트가 점차 작아지는 경우가 많음(**그래디언트 소실**)
    - 따라서 그냥 둘 경우 하위층의 연결 가중치가 업데이트되지 않을 것이며, 이 경우 제대로된 솔루션이 될 수 없음
    - 오히려 반대로 그래디언트가 너무 커져서 각 층이 비정상적으로 큰 가중치로 갱신되는 경우도 있음(**그래디언트 폭주**, 주로 RNN에서 발생)
- 이러한 이유는 명확하지 않지만, 의심되는 것들이 몇가지가 있음
    - 그 중에 하나는 로지스틱 시그모이드 활성화함수와 가중치 초기화방법(초기에는 가장 인기있었던 방법이 정규분포였음)의 조합이었음
        - 이 활성화 함수와 초기화 방식을 사용할 경우 각 층에서 출력 분산이 입력 분산보다 더 크다는 것이 알려짐
        - 결국 신경망의 위쪽으로 갈 수록 분산이 계속 커져 가장 높은 층에서는 활성화 함수가 0이나 1로 수렴하는 문제가 발생
        - 게다가 로지스틱 함수의 경우 평균이 0.5이기 떄문에 더 나빠짐(하이퍼볼릭은 평균이 0이므로 좀 더 괜찮았음)
            - 항상 로지스틱 함수가 양수를 뱉어내므로, 출력의 가중치 합이 입력보다 커질 가능성이 있었던 것이어씀!!
        - 로지스틱함수는 입력의 절댓값이 커지면 0이나 1로 수렴하는데, 이 때 그래디언트가 거의 없는 상태가 되어버림
            - 따라서 최상위층에서부터 역전파가 진행될 때 신호가 점점 약해져서 아래층에는 정보가 아예 도달하지 않는 문제가 발생한거심!

### 11.1.1. 글로럿과 He초기화
- 적절한 신호가 흐리기 위해서는 각 층의 출력과 분산이 입력에 대한 분산과 같아야하며, 역방향에서 층을 통과하기 전, 후의 그래디언트 분산이 동일해야함
    - 이는 이론적으로는 fan in과 fan out(입력층과 출력층의 연결개수)이 동일해야만 보장됨
    - 하지만 글로럿과 벤지오가 실전에서 매웆 잘 작동하는 것으로 입증된 대안을 제시
- **세이비어 초기화(글로럿 초기화)**
    - *logisitc activation, softmax, tanh를 쓸 때*
    - 각 층의 연결 가중치를 sigma^2 = 1/fan_avg 인 정규분포나 r = sqrt(3/fan_avg)일 때 -r ~ r의 균등분포로 무작위 초기화    
- **르쿤 초기화**
    - *SELU를 쓸 때*
    - 위에서 나온 식에서 fan_avg를 fain_in으로 바꿔줌
- 다른 논문들도 나왔지만, 일반적으로 분산의 스케일링이나 fan_avg를 쓰느냐 fan_in을 쓰느냐에 차이만 있음
- RELU활성화함수에 대한 초기화 전략을 논문 저자의 이름을 따서 **He 초기화**라고 부름

- 케라스는 기본적으로 균등분포의 글로럿 초기화를 사용함
    - 층을 만들 때, kernel_initializer = "he_uniform"을 인자로 대입하거나, kernel_initializer = "he_normal"을 인자로 대입하면 He 초기화도 사용할 수 있음
    - keras.layers.Dense(10, activation = "relu", kernel_initializer = 'he_normal')
- fan_in 대신에 fan_out기반의 He초기화를 사용하고 싶다면, keras.initializer 모듈의 VarianceScaling 클래스를 활용
    - he_avg_init = keras.initializers.VarianceScaling(scale = 2, mode = "fan_avg", distribution = "uniform")
    - keras.layers.Dense(10, activation = "sigmoid", kernel_initializer = he_agv_init)

### 11.1.2. 수렴하지 않는 활성화 함수
- 활성화 함수를 잘못 선택할 경우 그래디언트의 소실이나 폭주로 이어질 수 있다는 것을 발견
- 그 전에는 생물학적 뉴런의 행동을 본딴 시그모이드가 최선일 것이라 생각
    - 버뜨 다른 활성화 함수가 DNN에서 더 잘 작동함을 발견하게 되었고, 특히 ReLU가 그 대표
- **ReLU**의 경우 특정 양수값에 수렴하지 않는다는 큰 장점이 있었고, 연산도 단순하다는 큰 장점이 있었음
    - 물론 ReLU의 경우에도 문제가 있음
    - **죽은 ReLU** : 훈련하는 동안 일부 뉴런이 0 이외의 값을 출력하지 않음
    - 어떤 경우에는(특히 learning rate를 크게 잡아놓은 경우) 뉴런 절반이 죽어있을 수도 있음
    - 뉴런의 가중치가 바뀌어 훈련세트 내 모든 샘플에 대해 입력의 가중치 합이 음수가 되면 뉴런이 죽게 됨
        - 가중치 합이 음수 -> ReLU의 그래디언트 : 0 -> 따라서 경사하강법이 더이상 작동하지 않게됨
- 위 문제를 해결하기 위해 **LeakyReLU**와 같은 ReLU의 변종을 이용함
    - 1) LeakyReLU
        - LeakyReLU_a(Z) = max(az, z) : a가 새는 정도(leaky)의 정도를 정함
        - 새는 정도라는 것이 z < 0일 때의 함수의 기울기이며, 일반적으로 0.01로 지정
        - 그러면 모든 값을 0으로 반환하는 것이 아니기 때문에 LeakyReLU의 경우 죽을 가능성이 ㅇ벗어짐
        - 실제 연구에서는 a를 0.2정도로 크게 잡는 것이 좋은것으로 결과를 보이고 있음
    - 2) RReLU
        - Randomized Leaky ReLU
        - 규제의 역할도 수행하며, 꽤 잘 작동하는 것으로 보임
    - 3) PReLU
        - a를 학습시키는 Learky ReLU모델(Parametric Leaky ReLU)
        - 데이터가 적을 때 오버피팅이 되는 문제가 있었음
- **ELU**
    - Exponential Linear Unit이라는 2015년작 따끈한 신작 활성화함수
    - ReLU와 전반적으로 비슷한 특징을 보여주지만, 성능상으로는 ReLU류 함수들보다더 좋은 결과를 보여줌
    - if z <0, ELU_a(z) = a(exp(z) - 1), else z
    - 하지만 exponential함수를 사용하므로 ReLU류보다는 당연히 느릴 수밖에 없다는 점에 문제가 있음
        - 훈련하는 동안에는 지수함수로 연산하더라도, 수렴 속도가 워낙에 빨라 느린 계산이 상쇄되었지만, 테스트시에는 이 속도가 문제가될 수 있음
- **SELU**
    - Scaled ELU의 약자로 2017년 논문에서 소개됨
    - ELU의 변종이며, 완전 연결층만 쌓아 DNN을 만들고 모든 은닉층에 SELU를 적용할 경우, 네트워크가 자기정규화될 수 있다는 것을 보임
        - 즉, 각 층의 출력이 평균 0, 표준편차 1을 유지하려는 경향이 있다는 것
        - 이는 그래디언트 소실과 폭주 문제를 보여줄 수 있음
    - 종종 좋은 성능을 보이는데, 자기 정규화가 일어나려면 몇 가지 조건이 있는 것이 문제
        - 1) 입력 특성이 반드시 표준화(...? 애초에 여기서부터가...)
        - 2) 모든 은닉층의 가중치는 르쿤 정규분포로 초기화 되어야함
        - 3) 네트워크는 RNN이나 Skip Connection 등의 형태가 아닌 Sequential이어야함
- *책 추천 활성화함수 적용 순서 : SELU > ELU > LeakyReLU > ReLU > tanh > logisitic*

In [1]:
# #####LeakyRelU사용법 : 층을 우선 만들고, 적용하려는 층 뒤에 LeakyReLU층을 추가하듯 접근
# ##########PRelu도 사용법 동일

# model = keras.models.Sequential([
#     [...]
#     keras.layers.Dense(10, kernel_initializer = "he_normal")
#     keras.layers.LeakyReLU(alpha = 0.2)
#     [...]
# ])

In [2]:
# #####SELU사용법 : 층을 만들 때 activation = "selu", kernel_initializer = "lecun_normal"
# layer = keras.layers.Dense(10, activation = "selu", kernel_initializer = "lecun_normal")